# 07 - Hydrogen Chain

**Overview** 

This notebook guides you through ...  

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly > /dev/null
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
!pip install -q rdkit > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

In [ ]:
def tb_hamiltonian_1d(N, epsilon=0.0, t=-1.0, periodic=True):
    H = np.zeros((N, N), dtype=float)
    np.fill_diagonal(H, epsilon)
    off = t * np.ones(N - 1, dtype=float)
    H[np.arange(N-1), np.arange(1, N)] = off
    H[np.arange(1, N), np.arange(N-1)] = off
    if periodic and N > 2:
        H[0, -1] = t
        H[-1, 0] = t
    return H

def tb_dispersion_1d(k, epsilon=0.0, t=-1.0, a=1.0):
    return epsilon + 2.0 * t * np.cos(k * a)

def finite_k_open_chain(N, a=1.0):
    n = np.arange(1, N + 1)
    return n * np.pi / ((N + 1) * a)

def solve_finite_chain(N=20, epsilon=0.0, t=-1.0, periodic=True):
    H = tb_hamiltonian_1d(N, epsilon=epsilon, t=t, periodic=periodic)
    E, V = np.linalg.eigh(H)
    return E, V
import numpy as np

def _wrap_k(k, a=1.0):
    # map to (-pi/a, pi/a]
    return (k + np.pi/a) % (2*np.pi/a) - np.pi/a

def label_k_via_translation(E, V, a=1.0, energy_tol=1e-12):
    """
    Label periodic-chain eigenstates by signed k using the translation operator T.
    Also returns a rotated eigenvector matrix whose columns are chiral (traveling) states.

    Parameters
    ----------
    E : (N,) array of eigenvalues
    V : (N, N) eigenvectors (columns)
    a : lattice spacing
    energy_tol : tolerance to group degenerate energies

    Returns
    -------
    k : (N,) array of signed k in (-pi/a, pi/a]
    V_rot : (N, N) rotated eigenvectors with definite translation phase
    """
    N = V.shape[0]
    idx = np.argsort(E)
    E_sorted = E[idx]
    V_sorted = V[:, idx].copy()

    # group indices by (near-)degenerate energies
    groups = []
    start = 0
    for i in range(1, N+1):
        if i == N or not np.isclose(E_sorted[i], E_sorted[start], atol=energy_tol):
            groups.append((start, i))  # [start, i)
            start = i

    k_labels = np.empty(N, dtype=float)
    V_rot_sorted = V_sorted.copy()

    for (lo, hi) in groups:
        Vg = V_rot_sorted[:, lo:hi]          # N x g
        g = Vg.shape[1]

        # Build T in this subspace: S = <v_m | T | v_n> with (T psi)_j = psi_{j+1}
        # Implement T via roll by -1 along site axis
        TVg = np.roll(Vg, -1, axis=0)        # N x g
        S = Vg.conj().T @ TVg                # g x g

        # Diagonalize S: eigenvalues are e^{i k a}
        w, U = np.linalg.eig(S)
        # Rotate basis to chiral states
        Vg_rot = Vg @ U                      # N x g

        # Store rotated vectors back
        V_rot_sorted[:, lo:hi] = Vg_rot

        # Extract signed k from eigenvalues
        k_block = np.angle(w) / a            # in (-pi/a, pi/a]
        k_block = _wrap_k(k_block, a=a)

        # Pin exact special points
        # Gamma: keep exactly 0
        k_block[np.isclose(k_block, 0.0, atol=1e-14)] = 0.0
        # Zone edge (even N only): keep exactly +pi/a (convention)
        if N % 2 == 0:
            k_block[np.isclose(np.abs(k_block), np.pi/a, atol=1e-14)] = np.pi / a

        # Save
        k_labels[lo:hi] = k_block

    # undo sorting to match the original eigenvalue order
    inv = np.empty_like(idx)
    inv[idx] = np.arange(N)
    k_out = k_labels[inv]
    V_rot = V_rot_sorted[:, inv]

    return k_out, V_rot


**Problem** 

Lorem Ipsum ... 

**Model**

Lorem Ipsum ...

>Smart question?

Lorem Ipsum ...

**Questions**

Before you run any simulation, answer the following question(s):

1. Something
2. Seomthing else

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. Other questions

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
N = 100 # @param {type:"integer"}
epsilon = 0.0 # @param {type:"number"}
t = -1.0 # @param {type:"number"}
a = 1.0 # @param {type:"number"}
periodic = True # @param {type:"boolean"}
# periodic chain
E_per, V_per = solve_finite_chain(N=N, epsilon=epsilon, t=t, periodic=True)

k_signed, V_chiral = label_k_via_translation(E_per, V_per, a=1.0, energy_tol=1e-12)

# plot
order = np.argsort(k_signed)
k_grid = np.linspace(-np.pi, np.pi, 400)  # a=1
plt.figure()
plt.plot(k_grid, epsilon + 2*t*np.cos(k_grid), lw=2)
plt.plot(k_signed[order], E_per[order], 'o')
plt.xlabel('k (1/a)'); plt.ylabel('Energy (eV)')
plt.title('Ring: signed k from translation-operator phases')
plt.show()

In [ ]:
def plot_eigenstate(V, n, E=None, title_prefix='', mode='prob', sites=None, ax=None):
    N = V.shape[0]
    Nstates = V.shape[1]
    if not (-Nstates <= n < Nstates):
        raise ValueError(f"n must be in [-{Nstates}, {Nstates-1}], got {n}")
    n = n % Nstates  # enable negative indexing

    psi = V[:, n]
    x = np.arange(1, N+1) if sites is None else np.asarray(sites)

    if ax is None:
        fig, ax = plt.subplots()

    if mode == 'prob':
        y = np.abs(psi)**2
        ax.stem(x, y)
        ax.set_ylabel(r'$|\psi_n(i)|^2$')
    elif mode == 'amp':
        y = psi.real
        ax.stem(x, y)
        ax.axhline(0, lw=1)
        ax.set_ylabel(r'$\mathrm{Re}\,\psi_n(i)$')
    else:
        raise ValueError("mode must be 'prob' or 'amp'")

    ax.set_xlabel('Site i')
    if isinstance(E, (list, tuple, np.ndarray)):
        energy_txt = f'  (E = {E[n]:.4g})'
    elif isinstance(E, (float, int)):
        energy_txt = f'  (E = {float(E):.4g})'
    else:
        energy_txt = ''
    ax.set_title(f'{title_prefix}Eigenstate n={n+1}{energy_txt}')
    return ax

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(11,3), sharey=True)
plot_eigenstate(V_chiral, 5, E=E_per, ax=axes[0], mode='amp')
plot_eigenstate(V_chiral, 6, E=E_per, ax=axes[1], mode='amp')
plot_eigenstate(V_chiral, 7, E=E_per, ax=axes[2], mode='amp')
fig.tight_layout()

In [ ]:
n = 99
print(n)
x = np.arange(N)

plt.figure(figsize=(10,4))

# Raw (standing wave)
plt.subplot(1,2,1)
x = np.arange(1, N+1)
plt.plot(x, V_chiral[:, n].real, 'o-')
plt.title('Standing wave (real solver output)')
plt.xlabel('site'); plt.ylabel('ψ_i')

# Rotated (traveling wave)
plt.subplot(1,2,2)
plt.plot(x, V_chiral[:, n].real, 'o-', label='Re ψ')
plt.plot(x, V_chiral[:, n].imag, 'o-', label='Im ψ')
plt.legend()
plt.title(f'Traveling wave  k={k_signed[n]:.2f} (period {2*np.pi/abs(k_signed[n]):.1f} sites)')
plt.xlabel('site'); plt.ylabel('ψ_i')
plt.tight_layout()
plt.show()

In [ ]:
# @title Run and Visualize the Simulation  { display-mode: "form" }
# Initial conditions


# Run the simulation


**Homework Assignment**

Pick one (or more) of the following projects:
1. Modify the code to xxx
2. Modify the code to xxx
3. Modify the code to handle one of the following systems:
    * xxx
    * xxx 
    * xxx

For the modified code, answer the following questions:

NOTE: It is not necessary that the modified code produces an animation, but you should be able to visualize the results of the simulation in some way. 